In [1]:
!nvidia-smi

Sat Aug  5 11:07:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   45C    P0   107W / 300W |   1339MiB / 32508MiB |     37%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    #torch_dtype=torch.float16,
    #device_map="auto",
)

sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=2,
    num_return_sequences=4,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

I'm open to different genres, but I prefer shows with complex characters and engaging storylines.

Thanks!
Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Comment: Of course! If you enjoyed "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Sopranos" - This HBO series is a crime drama that explores the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.
2. "The Wire" - This HBO series is a gritty and realistic portrayal of the drug trade in Baltimore, exploring the impact it has on the city and its residents.
3. "Mad Men" - This AMC series is set in the 1960s and follows the lives of advertising executives on Madison Avenue, exploring themes of identity,
Result: I liked "Breaking Bad" and 

In [3]:
# creative prompts
import numpy as np
creative_prompts = ["Write a poem", "Tell me a joke", "Describe the feeling of love", "Write a story starting with 'Once upon a time...'",
                    "Tell a story about a dog", "Write a song", "Write a poem about a robot", "Invent an original recipe",
                    "Imagine a new object and describe what it looks like", "Imagine a new religion and describe its beliefs"]

# factual prompts
factual_prompts = ["What is the capital of France?", "How is H2O commonly known?", "What is the largest country in the world?", "How many days are in a year?",
                   "What is the largest planet in the solar system?", "What is the largest animal in the world?", "How do you say hello in Spanish?", "Who won the 2018 World Cup?",
                   "What is the biggest city in Europe?", "What is the largest country in Africa?"]





In [3]:
device = "cuda:1"
model = model.to(device)

In [6]:
from nltk.translate.bleu_score import sentence_bleu
def self_BLEU(sentences):
    scores = []
    for i in range(len(sentences)):
        score = sentence_bleu(sentences[:i] + sentences[i+1:], sentences[i])
        scores.append(score)
    print(scores)
    return np.mean(scores)

In [10]:
from typing import List, Callable
from typing import List, Literal, TypedDict, Callable

Role = Literal["system", "user", "assistant"]


class Message(TypedDict):
    role: Role
    content: str
    
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

#taken from https://github.com/viniciusarruda/llama-cpp-chat-completion-wrapper/blob/1c9e29b70b1aaa7133d3c7d7b59a92d840e92e6d/llama_cpp_chat_completion_wrapper.py#L21
def _llama2_format_messages(messages: List[Message], tokenizer_encode: Callable) -> List[int]:
    if messages[0]["role"] != "system":
        messages = [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT,
            }
        ] + messages
    messages = [
        {
            "role": messages[1]["role"],
            "content": B_SYS + messages[0]["content"] + E_SYS + messages[1]["content"],
        }
    ] + messages[2:]
    assert all([msg["role"] == "user" for msg in messages[::2]]) and all(
        [msg["role"] == "assistant" for msg in messages[1::2]]
    ), (
        "model only supports 'system', 'user' and 'assistant' roles, "
        "starting with 'system', then 'user' and alternating (u/a/u/a/u...)"
    )
    messages_tokens: List[int] = sum(
        [
            tokenizer_encode(
                f"{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()} ",
                bos=True,
                eos=True,
            )
            for prompt, answer in zip(
                messages[::2],
                messages[1::2],
            )
        ],
        [],
    )
    assert messages[-1]["role"] == "user", f"Last message must be from user, got {messages[-1]['role']}"
    messages_tokens += tokenizer_encode(
        f"{B_INST} {(messages[-1]['content']).strip()} {E_INST}",
        bos=True,
        eos=False,
    )
    return messages_tokens

In [14]:
new_message = Message(role="user", content="Write a poem")


_llama2_format_messages([new_message], tokenizer.encode)

TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'bos'

In [17]:
prompt = """<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Write a poem [/INST]"""

input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
output_ids = model.generate(input_ids, max_length=300, pad_token_id=tokenizer.eos_token_id, do_sample=True, temperature=1.2, num_return_sequences=5)

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Write a poem [/INST]  Of course, I'd be happy to help you with a poem! Here's a poem that I came up with:
Safe and respectful, I'll always be
A helpful assistant, just like magic to see
I'll answer your questions, both big and small
With honesty and kindness, I'll stand tall

I'll never share harmful or false information
My answers will always be socially unbiased and positive in nature
I'll never promote racist or sexist views
I'll always strive to be safe and 

In [6]:
print(tokenizer.decode(output_ids[1], skip_special_tokens=True))


[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Write a poem [/INST]  Of course, I'd be happy to help you with a poem! Here's a poem that I hope you'll find helpful and respectful:
Safe and sound, I'll answer your call,
With honesty and integrity, I stand tall.



In [7]:


# each temperature and for each prompt, generate n_generations samples
temperaturesa= np.linspace(0.4, 1.4, 11)
n_generations = 25

bleu_scores_creative = np.zeros((len(temperatures), len(creative_prompts)))
bleu_scores_factual = np.zeros((len(temperatures), len(factual_prompts)))

for i, prompt in enumerate(creative_prompts):
    for j, temperature in enumerate(temperatures):
        samples = []
        for _ in range(n_generations):
            input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
            sample = model.generate(input_ids, temperature=temperature, max_length=50)
            sample = tokenizer.decode(sample[0], skip_special_tokens=True)
            print(sample)
            samples.append(sample)
        print(f"Prompt: {prompt}")
        print(f"Temperature: {temperature}")
        print(f"Samples: {samples}")
        # compute self-BLEU
        bleu_scores_creative[j, i] = self_BLEU(samples)


        


Write a poem about the beauty of the night sky

The night sky is a canvas of wonder,
A tapestry of stars and moonlight,
A celestial display of beauty,
That fills the heart with delight.
Write a poem about the beauty of the night sky

The night sky is a canvas of wonder,
A tapestry of stars and moonlight,
A celestial display of beauty,
That fills the heart with delight.
Write a poem about the beauty of the night sky

The night sky is a canvas of wonder,
A tapestry of stars and moonlight,
A celestial display of beauty,
That fills the heart with delight.
Write a poem about the beauty of the night sky

The night sky is a canvas of wonder,
A tapestry of stars and moonlight,
A celestial display of beauty,
That fills the heart with delight.
Write a poem about the beauty of the night sky

The night sky is a canvas of wonder,
A tapestry of stars and moonlight,
A celestial display of beauty,
That fills the heart with delight.
Write a poem about the beauty of the night sky

The night sky is a can

KeyboardInterrupt: 

In [10]:
str1 = "Paris"
str2 = "ParisParis"

# compute BLEU score using nltk
from nltk.translate.bleu_score import sentence_bleu
sentence_bleu([str1], str2)


0.392814650900513

0.41113361690051975